In [ ]:
import requests
from PIL import Image
import gradio as gr
import io

# Custom Vision Prediction 키와 URL
PREDICTION_KEY = "48fBztGiqcoH4PUUK23qmvxTBMmu31mgQ4JmJN1EDeCWNYw2QAtQJQQJ99BEACL93NaXJ3w3AAAIACOGelyc"
ENDPOINT_URL = "https://neptun91cv20250526-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/c387e65a-5db0-4e24-8b0a-3c049157dcc2/detect/iterations/Iteration1/image"

headers = {"Prediction-Key": PREDICTION_KEY, "Content-Type": "application/octet-stream"}


def predict_with_api(image: Image.Image):
    # 이미지를 바이너리로 변환
    buf = io.BytesIO()
    image.save(buf, format="JPEG")
    byte_data = buf.getvalue()

    # API 호출
    response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)
    predictions = response.json()["predictions"]

    # 가장 높은 확률의 예측 추출
    top_prediction = max(predictions, key=lambda x: x["probability"])
    label = top_prediction["tagName"]
    probability = top_prediction["probability"]

    return f"{label} ({probability*100:.2f}%)"


# Gradio 인터페이스 정의
interface = gr.Interface(
    fn=predict_with_api,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(),
    title="Custom Vision Image Classifier",
    description="Upload an image to see the prediction from your Custom Vision model.",
)

interface.launch()